## Library imports

In [6]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import morphology
from skimage.morphology import opening
from skimage import filters
from functools import partial 
import pandas as pd
from os import listdir
import os
from os.path import isfile, join
import fyp2021p3_group00_functions as util
import seaborn as sns
import cv2, numpy as np
from sklearn.cluster import KMeans
from matplotlib import rcParams
from PIL import Image
import warnings

# Define functions

In [7]:
# this code append the rgb values into a pandas dataframe

def color_normalizer(arr, name):
    #print("function")
    #print(arr)
    counter = 1
    for (percent, color) in arr:
        #print("picture is ",name,"  round ",counter)
        c1 = int(color[0])
        c2 = int(color[1])
        c3 = int(color[2])
        #print("colors are\n")
        #print(c1,"",c2,"",c3)
        #print(percent)
        #print(name)
        index = pics[pics['image_id'] == name].index.values
        rowIndex = pics.index[index]
        rgb_str = str(c1) + "," + str(c2) + "," + str(c3)
        #print("this will be the rgb_str\n")
        #print(rgb_str)
        pics.loc[rowIndex, 'colorRGB_'+str(counter)] = rgb_str
        pics.loc[rowIndex, 'occurence_of_colour_percent_'+str(counter)] = percent*100
        #pics.loc[rowIndex, 'trial'] = color
        #pics[name] = color
        counter = counter + 1
    None
    
def dominant_colors(cluster, centroids, name):
    # Get the number of different clusters, create histogram, and normalize
    labels = np.arange(0, len(np.unique(cluster.labels_)) + 1)
    (hist, _) = np.histogram(cluster.labels_, bins = labels)
    hist = hist.astype("float")
    #print(hist)
    hist /= hist.sum()
    # Create frequency rect and iterate through each cluster's color and percentage
    rect = np.zeros((50, 300, 3), dtype=np.uint8)
    colors = sorted([(percent, color) for (percent, color) in zip(hist, centroids)])
    start = 0
    color_normalizer(colors, name) 
    for (percent, color) in colors:
        end = start + (percent * 300)
        cv2.rectangle(rect, (int(start), 0), (int(end), 50), \
                      color.astype("uint8").tolist(), -1)
        start = end
    return rect

# Read in raw datasets

In [8]:
pics = pd.read_csv("../data/example_ground_truth.csv", sep = ",")
pics_to_process = pd.read_csv("../data/processed/dominant_colors_raw.csv", sep = ",")

# Search the 4 most dominant color of each picture

Loop through a 25% smaller pictures. Code makes a pandas dataframe with 4 - 4 column (RGB code in string than occurance in %)

In [9]:
WARN = input("This operation takes several minutes, the raw dataset is already exist. Do you wish to continue: (Yes/No) ")

if WARN.lower().startswith("y"):
    print("\n---It takes time --- \n")
    for i in pics["image_id"]:
        #print(i)
        # Alternative: replace the non-lesion pixels
        im = plt.imread('../data/example_image/sm/'+i+'.jpg')
        #plt.imshow(im)
        img2 = im.copy()
        mask=plt.imread('../data/example_segmentation/sm/'+i+'_segmentation.png')
        img2[mask==0] = 0
        #plt.imshow(img2)
        image = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
        reshape = img2.reshape((img2.shape[0] * img2.shape[1], 3))
        # Find and display most dominant colors
        cluster = KMeans(n_clusters=5).fit(reshape)
        visualize = dominant_colors(cluster, cluster.cluster_centers_, i)
    display(pics)
    pics.to_csv('../data/processed/dominant_colors_raw.csv')
else:
    print("Cancelled operation")

This operation takes several minutes, the raw dataset is already exist. Do you wish to continue: (Yes/No) no
Cancelled operation


In [10]:
# highest element check, also for a good masks
for i in range(len(pics_to_process["image_id"])):
    txt = pics_to_process["colorRGB_5"][i]
    x = txt.split(",")
    #print(pics_to_process["image_id"][i]," ",x)
no_adjsutment_examples_column_5 = ["ISIC_0009995","ISIC_0014688","ISIC_0014712","ISIC_0014937","ISIC_0015062","ISIC_0015256","ISIC_0015372"]

## Delete max values if they are the black areas and adjust the remain 4 dominant colors

In [12]:
for i in range(len(pics_to_process["image_id"])):
    txt = pics_to_process["colorRGB_5"][i]
    x = txt.split(",")
    sumy = 0
    for y in range(len(x)):
        sumy = sumy + int(x[y])
    if sumy > 0:
        print(pics_to_process["image_id"][i]," ",x)
no_adjsutment_examples_column_5 = ["ISIC_0009995","ISIC_0014688","ISIC_0014712","ISIC_0014937","ISIC_0015062","ISIC_0015256","ISIC_0015372"]

ISIC_0009995   ['164', '137', '104']
ISIC_0014688   ['171', '119', '83']
ISIC_0014712   ['115', '92', '87']
ISIC_0014937   ['155', '98', '74']
ISIC_0015062   ['119', '90', '96']
ISIC_0015256   ['201', '110', '89']
ISIC_0015372   ['71', '30', '33']


In [13]:
# rearrange the non-black colours in column 5 
for i in range(len(pics_to_process["image_id"])):
    pic_id = pics_to_process["image_id"][i]
    if any(t == pic_id for t in no_adjsutment_examples_column_5):
        index = pics_to_process[pics_to_process['image_id'] == pic_id].index.values
        rowIndex = pics_to_process.index[index]
        pics_to_process.loc[rowIndex, 'adjusted_bool'] = 0
    else:
        index = pics_to_process[pics_to_process['image_id'] == pic_id].index.values
        rowIndex = pics_to_process.index[index]
        pics_to_process.loc[rowIndex, 'adjusted_bool'] = 1
        remains = 100 - pics_to_process["occurence_of_colour_percent_5"][i]
        for i in range(1,4+1):
            new_occ_percent = (100/remains)*pics_to_process['occurence_of_colour_percent_'+str(i)]        
            pics_to_process.loc[rowIndex, 'adjusted_percent'+str(1+i)] = new_occ_percent
            None

In [14]:
# final daztaset about dominant colours
# it gives back lot of errors, bnut still functional
pics_raw = pd.read_csv("../data/processed/dominant_colors_adjusted.csv", sep = ",")
pics_raw.reset_index()
pics_raw["RGB1"] = 0
pics_raw["RGB2"] = 0
pics_raw["RGB3"] = 0
pics_raw["RGB4"] = 0
pics_raw["RGB1_occ"] = 0
pics_raw["RGB2_occ"] = 0
pics_raw["RGB3_occ"] = 0
pics_raw["RGB4_occ"] = 0
for i in range(len(pics_raw["image_id"])):
    if pics_raw["adjusted_bool"][i] == 1:
        t = pics_raw["colorRGB_1"][i]
        pics_raw["RGB1"][i] = t
        pics_raw["RGB2"][i] = pics_raw["colorRGB_2"][i]
        pics_raw["RGB3"][i] = pics_raw["colorRGB_3"][i]
        pics_raw["RGB4"][i] = pics_raw["colorRGB_4"][i]
        pics_raw["RGB1_occ"][i] = pics_raw["adjusted_percent2"][i]
        pics_raw["RGB2_occ"][i] = pics_raw["adjusted_percent3"][i]
        pics_raw["RGB3_occ"][i] = pics_raw["adjusted_percent4"][i]
        pics_raw["RGB4_occ"][i] = pics_raw["adjusted_percent5"][i]
    else:
        pics_raw["RGB1"][i] = pics_raw["colorRGB_2"][i]
        pics_raw["RGB2"][i] = pics_raw["colorRGB_3"][i]
        pics_raw["RGB3"][i] = pics_raw["colorRGB_4"][i]
        pics_raw["RGB4"][i] = pics_raw["colorRGB_5"][i]
        pics_raw["RGB1_occ"][i] = pics_raw["occurence_of_colour_percent_2"][i]
        pics_raw["RGB2_occ"][i] = pics_raw["occurence_of_colour_percent_3"][i]
        pics_raw["RGB3_occ"][i] = pics_raw["occurence_of_colour_percent_4"][i]
        pics_raw["RGB4_occ"][i] = pics_raw["occurence_of_colour_percent_5"][i]
        
pics_raw = pics_raw.drop(['colorRGB_1','colorRGB_2','colorRGB_3','colorRGB_4','colorRGB_5'],axis=1)
pics_raw = pics_raw.drop(['occurence_of_colour_percent_1','occurence_of_colour_percent_2','occurence_of_colour_percent_3','occurence_of_colour_percent_4','occurence_of_colour_percent_5'],axis=1)
pics_raw = pics_raw.drop(['adjusted_percent2','adjusted_percent3','adjusted_percent4','adjusted_percent5'],axis=1)
pics_raw.to_csv('../data/processed/dominant_colors_final_dataset.csv')

<ipython-input-14-333a28cdf870>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pics_raw["RGB1"][i] = t
c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-14-333a28cdf870>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pics_raw["RGB2"][i] = pics_raw["colorRGB_2"][i]
<ipython-input-14-333a28cdf870>:18: SettingWithCopyWarn

PermissionError: [Errno 13] Permission denied: '../data/processed/dominant_colors_final_dataset.csv'

## Merge datasets

In [15]:
dominant_color = pd.read_csv("../data/processed/dominant_colors_final_dataset.csv", sep = ",")
border_symmetry = pd.read_csv("../data/processed/border_symmetry.csv", sep = ",")
border_symmetry = border_symmetry.drop(['image_id'],axis=1)
dominant_color = dominant_color.drop(['Unnamed: 0','Unnamed: 0.1','adjusted_bool'],axis=1)
final_df = pd.concat([dominant_color, border_symmetry], axis=1)
final_df = final_df.drop(['Unnamed: 0.1.1'],axis=1)
final_df.to_csv('../data/processed/final_df.csv')

,image_id,melanoma,seborrheic_keratosis,RGB1,RGB2,RGB3,RGB4,RGB1_occ,RGB2_occ,RGB3_occ,RGB4_occ,border_diff_percent,rotation_amount_deg
0,ISIC_0001769,0.0,0.0,"180,131,103","170,111,78","147,78,47","159,93,61",16,24,26,32,60.366446,70
1,ISIC_0001852,0.0,0.0,"144,79,44","163,109,80","120,49,23","87,27,15",15,19,19,45,31.599922,90
2,ISIC_0001871,0.0,0.0,"171,115,75","184,138,106","189,151,129","179,127,91",15,25,27,32,48.707232,120
3,ISIC_0003462,0.0,0.0,"162,105,69","172,128,100","180,150,132","149,84,48",18,19,30,31,31.444401,90
4,ISIC_0003539,0.0,0.0,"113,36,17","152,72,32","173,106,64","182,129,93",12,21,25,40,64.507193,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,ISIC_0015443,0.0,0.0,"160,118,103","121,59,26","133,81,52","144,98,75",19,20,28,30,382.372667,130
146,ISIC_0015445,0.0,0.0,"88,28,20","216,141,128","130,73,64","171,110,100",19,23,24,32,463.209565,90
147,ISIC_0015483,0.0,0.0,"87,54,42","189,147,131","124,90,77","156,120,106",17,24,27,30,366.120723,90
148,ISIC_0015496,0.0,0.0,"154,75,51","71,27,16","216,116,91","243,169,144",20,25,27,27,416.412325,50
